In [32]:
import pandas as pd
import os
import numpy as np
from scipy import stats
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import  model_from_json 
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
import warnings
warnings.filterwarnings('ignore')

files = os.listdir("accel")

In [42]:
def get_segments(dataframe, step,segment_size =128):
    
    segments = []
    labels = []
    
            
    for i in range(0,  dataframe.shape[0]- segment_size, step):  

        xs = dataframe['x-axis'].values[i: i + segment_size]
        ys = dataframe['y-axis'].values[i: i + segment_size]
        zs = dataframe['z-axis'].values[i: i + segment_size]
        label = stats.mode(dataframe['activity'][i: i + segment_size])[0]
        segments.append([xs, ys, zs])
        labels.append(label[0])
        
    #reshape the segments which is (list of arrays) to a list
    print(np.array(segments).shape)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, 1, segment_size,3)

    labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    return reshaped_segments , np.array(labels)

def loading_files(txt_files, start_file , end_file):
    columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
    df = pd.concat([pd.read_csv('accel/'+item, names=columns) for item in txt_files[start_file:end_file]]).reset_index(drop=True)

    df = df.dropna()
    df['z-axis'] = df['z-axis'].str.replace(';', '')
    df['z-axis'] = df['z-axis'].apply(lambda x:float(x))

    df = df[df['timestamp'] != 0]
    df = df.sort_values(by = ['user', 'timestamp'], ignore_index=True)

    walk = df[df['activity'] == 'A'  ].reset_index(drop=True)
    run = df[df['activity'] == 'B'  ].reset_index(drop=True)
    sit = df[df['activity'] == 'D'  ].reset_index(drop=True)
    stand = df[df['activity'] == 'E'  ].reset_index(drop=True)

    df = pd.concat([walk,run,sit,stand]).reset_index(drop=True)
    return df

In [43]:
txt_files = [file for file in files]
df = loading_files(txt_files ,0,20)
df_test =loading_files(txt_files ,21,26)

In [5]:
#A = 1
#B = 2
#D = 3
#E = 4
df.head(10)

,user,activity,timestamp,x-axis,y-axis,z-axis
0,1600,A,90426708196641,7.091625,-0.591667,8.195502
1,1600,A,90426757696641,4.972757,-0.158317,6.696732
2,1600,A,90426807196641,3.253720,-0.191835,6.107758
3,1600,A,90426856696641,2.801216,-0.155922,5.997625
4,1600,A,90426906196641,3.770868,-1.051354,7.731027
5,1600,A,90426955696641,4.661511,0.169689,9.684695
6,1600,A,90427005196641,6.145916,0.832883,11.003901
7,1600,A,90427054696641,7.259220,-0.792780,11.485135
8,1600,A,90427104196641,7.354988,-2.535759,11.454011
9,1600,A,90427153696641,7.319075,-2.700959,11.461193


In [45]:
train_segments , train_labels = get_segments(df,50,100)

test_dictionary = {}
for i in df_test['user'].unique():
    temp_df = df_test[df_test['user'] == i]
    test , testL = get_segments(temp_df,50,100)
    test_dictionary[i] = (test , testL)

(7794, 3, 100)
(289, 3, 100)
(397, 3, 100)
(287, 3, 100)
(287, 3, 100)
(286, 3, 100)


In [46]:
print(train_segments.shape, train_labels.shape)

(7794, 1, 100, 3) (7794, 4)


In [47]:
initializer = tf.keras.initializers.Orthogonal()

In [48]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense

model = Sequential()
model.add(Conv2D(16, (1, 1), name="conv2d_1", input_shape=(1, 100, 3), padding="same",kernel_initializer=initializer,bias_initializer='zeros'))
model.add(MaxPooling2D((1, 2), name="max_pooling2d_1"))

model.add(Conv2D(64, (1, 1), name="conv2d_2", padding="same",kernel_initializer=initializer,bias_initializer='zeros'))
model.add(MaxPooling2D((1, 2), name="max_pooling2d_2"))

model.add(Dropout(0.5, name="dropout_1"))

model.add(Conv2D(256, (1, 12), name="conv2d_3",kernel_initializer=initializer,bias_initializer='zeros'))
model.add(MaxPooling2D((1, 2), name="max_pooling2d_3"))

model.add(Conv2D(512, (1, 1), name="conv2d_4",kernel_initializer=initializer,bias_initializer='zeros'))
model.add(MaxPooling2D((1, 2), name="max_pooling2d_4"))

model.add(Dropout(0.5, name="dropout_2"))

model.add(GlobalAveragePooling2D(name="global_average_pooling"))

model.add(Dense(50, activation='relu', name="dense_1",kernel_initializer=initializer,bias_initializer='zeros'))
model.add(Dense(4, activation='relu', name="dense_2",kernel_initializer=initializer,bias_initializer='zeros'))

In [49]:
for layer in model.layers:
    print(layer.output_shape , layer.name)

(None, 1, 100, 16) conv2d_1
(None, 1, 50, 16) max_pooling2d_1
(None, 1, 50, 64) conv2d_2
(None, 1, 25, 64) max_pooling2d_2
(None, 1, 25, 64) dropout_1
(None, 1, 14, 256) conv2d_3
(None, 1, 7, 256) max_pooling2d_3
(None, 1, 7, 512) conv2d_4
(None, 1, 3, 512) max_pooling2d_4
(None, 1, 3, 512) dropout_2
(None, 512) global_average_pooling
(None, 50) dense_1
(None, 4) dense_2


In [50]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 1, 100, 16)        64        
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 1, 50, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 50, 64)         1088      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 1, 25, 64)         0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 1, 25, 64)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 1, 14, 256)       

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_segments, train_labels,batch_size=64,epochs=20)

Epoch 1/20
122/122 [==============================] - 6s 37ms/step - loss: 0.4684 - accuracy: 0.8002
Epoch 2/20
122/122 [==============================] - 4s 34ms/step - loss: 0.2460 - accuracy: 0.9039
Epoch 3/20
122/122 [==============================] - 4s 36ms/step - loss: 0.2211 - accuracy: 0.9095
Epoch 4/20
122/122 [==============================] - 4s 34ms/step - loss: 0.1778 - accuracy: 0.9240
Epoch 5/20
122/122 [==============================] - 4s 35ms/step - loss: 0.1796 - accuracy: 0.9267
Epoch 6/20
122/122 [==============================] - 4s 35ms/step - loss: 0.1750 - accuracy: 0.9244
Epoch 7/20
122/122 [==============================] - 4s 35ms/step - loss: 0.1519 - accuracy: 0.9326
Epoch 8/20
122/122 [==============================] - 5s 37ms/step - loss: 0.1612 - accuracy: 0.9296
Epoch 9/20
122/122 [==============================] - 4s 35ms/step - loss: 0.1490 - accuracy: 0.9334
Epoch 10/20
122/122 [==============================] - 4s 33ms/step - loss: 0.1680 - accura

In [31]:
for key in test_dictionary.keys():
    test, testL = test_dictionary[key]
    print("Result for User ",key)
    model.evaluate(test, testL)

Result for User  1602
10/10 [==============================] - 0s 8ms/step - loss: 0.1132 - accuracy: 0.9412
Result for User  1628
13/13 [==============================] - 0s 6ms/step - loss: 2.4750 - accuracy: 0.4962
Result for User  1632
9/9 [==============================] - 0s 9ms/step - loss: 0.0750 - accuracy: 0.9861
Result for User  1635
9/9 [==============================] - 0s 7ms/step - loss: 0.2253 - accuracy: 0.9094
Result for User  1643
9/9 [==============================] - 0s 8ms/step - loss: 0.0263 - accuracy: 0.9895


In [159]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")